# WindAI Competition Submission Notebook

## Prepare Local Dataset

In [ ]:
from wind.preprocess import prepare_local_data

prepare_local_data.main()

## Make Local Predictions

In [ ]:
from wind.model import pred_local_windpower

pred_local_windpower.main()

## Prepare Area Dataset

In [ ]:
from wind.preprocess import prepare_area_data

prepare_area_data.main()

## Make Area Predictions

In [ ]:
from wind.model import pred_area_bayesian

prepare_area_data.main()